In [1]:
using DataFrames, DataFramesMeta, LinearAlgebra, CSV, Distributions, Optim, Revise

In [2]:
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__)
using BLPmodule

In [3]:
I = 50;                # Number of consumers
J = 10;                 # Number of firms
K = 1;                  # Product characteristics
nT = 5;                # Number of markets
β = [2, -1];        # Preferences
varζ = 5;               # Variance of the random taste
rangeJ = [2, 6];        # Min and max firms per market
rangeB = [0, 1];        # Min and max consumers per market (e.g. census block)
varX = 1;               # Variance of X
varξ = 2;               # Variance of xi

In [4]:
using BLPmodule
# Simulate
df = BLPmodule.simulate_data(I, J, β, nT, rangeJ, rangeB, varζ, varX, varξ)
# Retrieve data
T = Int.(df.t)
X_ = [df.x1 df.p]
# s_ = df.s
q_j_ = df.q_j
s0_ = df.s0
IV_ = [df.x1 df.w]
Z_ = [df.x1 df.z1]
println(names(df))

["j", "w", "p", "s", "s0", "pr", "q", "q0", "B", "x1", "z1", "t", "q_j"]


In [5]:
# Draw shocks (less)
ζ_ = [rand(Normal(0,1), 1, I); zeros(K, I)]

2×50 Matrix{Float64}:
 -1.55982  -1.69464  1.87477  -0.471827  …  -1.68254  -1.23604  1.14481
  0.0       0.0      0.0       0.0           0.0       0.0      0.0

In [6]:
# Minimize GMM objective function
varζ_ = optimize(x -> BLPmodule.GMM(q_j_, X_, Z_, ζ_, T, IV_, x[1])[1], [2.0], LBFGS()).minimizer[1]

2.0

In [7]:
β_blp = BLPmodule.GMM(q_j_, X_, Z_, ζ_, T, IV_, varζ_)[2]
print("Estimated BLP coefficients: $β_blp")

Estimated BLP coefficients: [NaN, NaN]

In [8]:
# # Compute logit estimate
# y = log.(df.s) - log.(df.s0)
# β_logit = inv(IV_' * X_) * (IV_' * y)
# print("Estimated logit coefficients: $β_logit")